In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
from bs4 import BeautifulSoup


import warnings

warnings.filterwarnings("ignore")

In [2]:
url = "https://en.wikipedia.org/wiki/Radiohead"

In [3]:
page = requests.get(url)

In [5]:
soup = BeautifulSoup(page.text,"html.parser")
paragraphs = soup.find_all("p")
for paragraph in paragraphs:
    
    text_list = [paragraph.get_text() for paragraph in paragraphs]
print(text_list)
type(text_list)
    

['\n', "Radiohead are an English rock band formed in Abingdon, Oxfordshire, in 1985. They comprise Thom Yorke (vocals, guitar, piano, keyboards); brothers Jonny Greenwood (guitar, keyboards, other instruments) and Colin Greenwood (bass); Ed O'Brien (guitar, backing vocals); and Philip Selway (drums, percussion). They have worked with the producer Nigel Godrich and the cover artist Stanley Donwood since 1994. Radiohead's experimental approach is credited with advancing the sound of alternative rock.\n", 'Radiohead signed to EMI in 1991 and released their debut album, Pablo Honey, in 1993. Their debut single, "Creep", was a worldwide hit, and their popularity and critical standing rose with The Bends in 1995. Their third album, OK Computer (1997), is acclaimed as a landmark record and one of the greatest albums in popular music, with complex production and themes of modern alienation. Their fourth album, Kid A (2000), marked a dramatic change in style, incorporating influences from elect

list

In [ ]:
import spacy
import re
# Load the English NLP model from spaCy
nlp = spacy.load("en_core_web_sm")
def preprocess(text_list):
    full_text = ''.join(text_list)
    print(full_text)
    import spacy
    # Load the English NLP model from spaCy
    nlp = spacy.load("en_core_web_sm")

    # Process the full_text with spaCy
    doc = nlp(full_text)

    # Remove stop words
    filtered_tokens = [token.lemma_.lower() for token in doc if not token.is_stop and not token.is_punct and token.text != ',']
    filtered_tokens = [token for token in filtered_tokens if re.match("^[a-zA-Z]+$",token)]


    return filtered_tokens

In [7]:
preprocessed_text = preprocess(text_list)
print(type(preprocessed_text))


Radiohead are an English rock band formed in Abingdon, Oxfordshire, in 1985. They comprise Thom Yorke (vocals, guitar, piano, keyboards); brothers Jonny Greenwood (guitar, keyboards, other instruments) and Colin Greenwood (bass); Ed O'Brien (guitar, backing vocals); and Philip Selway (drums, percussion). They have worked with the producer Nigel Godrich and the cover artist Stanley Donwood since 1994. Radiohead's experimental approach is credited with advancing the sound of alternative rock.
Radiohead signed to EMI in 1991 and released their debut album, Pablo Honey, in 1993. Their debut single, "Creep", was a worldwide hit, and their popularity and critical standing rose with The Bends in 1995. Their third album, OK Computer (1997), is acclaimed as a landmark record and one of the greatest albums in popular music, with complex production and themes of modern alienation. Their fourth album, Kid A (2000), marked a dramatic change in style, incorporating influences from electronic music,

In [8]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

tokenized_corpus = [word_tokenize(sentence.lower()) for sentence in preprocessed_text]
tokenized_corpus = [tokens for tokens in tokenized_corpus if tokens]


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Hp\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [9]:
import gensim.downloader as api
model = api.load("glove-wiki-gigaword-300")
model.most_similar("radiohead")







[('coldplay', 0.6285437345504761),
 ('r.e.m.', 0.540037214756012),
 ('metallica', 0.5379277467727661),
 ('u2', 0.5235767960548401),
 ('björk', 0.5166013836860657),
 ('frontman', 0.5074184536933899),
 ('aerosmith', 0.5065386891365051),
 ('weezer', 0.49893084168434143),
 ('wilco', 0.4829169809818268),
 ('bandmates', 0.4812430441379547)]

In [15]:
from collections import Counter

# Flatten the tokenized_corpus
flattened_corpus = [word for sentence in tokenized_corpus for word in sentence]

# Filter tokens that exist in the model's vocabulary
filtered_tokens = [word for word in flattened_corpus if word in model.key_to_index]

# Count the occurrences of each word
word_counts = Counter(filtered_tokens)
word_counts

Counter({'radiohead': 216,
         'album': 97,
         'release': 83,
         'yorke': 64,
         'music': 56,
         'band': 47,
         'greenwood': 41,
         'record': 39,
         'song': 36,
         'work': 33,
         'tour': 33,
         'rock': 30,
         'jonny': 30,
         'include': 28,
         'say': 25,
         'kid': 24,
         'influence': 24,
         'new': 23,
         'play': 22,
         'perform': 22,
         'number': 22,
         'single': 20,
         'year': 20,
         'early': 19,
         'godrich': 18,
         'emi': 18,
         'debut': 18,
         'time': 18,
         'ok': 17,
         'computer': 17,
         'film': 17,
         'artist': 16,
         'good': 16,
         'follow': 16,
         'rainbow': 15,
         'uk': 15,
         'recording': 15,
         'performance': 15,
         'begin': 15,
         'video': 15,
         'live': 15,
         'chart': 14,
         'member': 14,
         'describe': 14,
         'se

In [16]:
word_to_index = {}
index_to_word = {}
for idx, word in enumerate(sorted(set(filtered_tokens))):
    word_to_index[word] = idx + 1  # start indexing from 1 (reserve 0 for padding)
    index_to_word[idx + 1] = word

In [17]:
vocab_size = len(word_to_index) + 1 # for padding token at index 0
embedding_dim = 300
embedding_matrix = np.zeros((vocab_size,embedding_dim))

for word, idx in word_to_index.items():
    if word in model:
        embedding_matrix[idx] = model[word]
    else:
        embedding_matrix[idx] = np.random.normal(scale=0.6,size = (embedding_dim))

In [ ]:
input_seqs, output_words = [], []

for i in range(len(flattened_corpus) - 3):  
    input_seq = flattened_corpus[i:i+3]
    output_word = flattened_corpus[i+3]

    # Check if all words in input_seq and output_word exist in word_to_index
    if all(word in word_to_index for word in input_seq + [output_word]):
        input_seqs.append([word_to_index[word] for word in input_seq])
        output_words.append(word_to_index[output_word])

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Ensure input_seq is a list of lists of integers
input_seq = [[word_to_index[word] for word in seq if word in word_to_index] for seq in input_seqs]

# Pad the sequences
X = pad_sequences(input_seq, maxlen=3, padding='pre')
y= np.array(output_words)


In [37]:
from keras.models import Sequential
from keras.layers import Embedding, Dense, LSTM
from keras.callbacks import EarlyStopping

In [38]:
nn = Sequential()

#"Embedding: Maps words to dense vectors in lower-dimensional space.
nn.add(Embedding(input_dim=vocab_size,
                    output_dim=300,  # Match the second dimension of embedding_matrix
                    weights=[embedding_matrix],
                    input_length=3,
                    trainable=False))

#"LSTM: Recurrent neural network for modeling sequential data effectively.
nn.add(LSTM(1000,return_sequences=True))
nn.add(LSTM(1000))

#Dense: Fully connected layer in neural networks for feature transformation
nn.add(Dense(1000,activation="relu"))

# output layer
nn.add(Dense(vocab_size,activation="softmax"))

In [39]:
nn.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)         │ ?                      │       459,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_4 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_5 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 459,000 (1.75 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 459,000 (1.75 MB)

In [40]:
early_stop = EarlyStopping(monitor="loss",patience=3)


In [41]:
nn.compile(loss="categorical_crossentropy",optimizer="adam",metrics=["accuracy"])


In [ ]:
from tensorflow.keras.utils import to_categorical

# Convert integer targets to one-hot encoding
y_categorical = to_categorical(y, num_classes=vocab_size)


# Fit the model
hist = nn.fit(X, y_categorical, epochs=3, callbacks=[early_stop])


Epoch 1/3


ValueError: Attr 'Toutput_types' of 'OptionalFromValue' Op passed list of length 0 less than minimum 1.